In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from time import time, sleep

from utils import *
from jigsaws_utils import *
from custom_pca import custom_pca
from video_loader import VideoLoader
from autoencoders import *
from physionet_pytorch.utils.get_data import *

import warnings
warnings.simplefilter('ignore', category=np.ComplexWarning)

def cross_predict(model, gram_matrix, y, nsplits=5, d_transform=None):
    nsample = len(y)
    nsample_per_group = int(nsample/nsplits)
    repeats = [nsample_per_group]*(nsplits-1)
    repeats = repeats + [nsample-np.sum(repeats)]
    groups = np.repeat(np.arange(nsplits), repeats)
    kfolder = GroupKFold(n_splits=nsplits)

    all_preds = []
    all_true = []
    for i, (train_index, test_index) in enumerate(kfolder.split(y, groups=groups)):
        if d_transform is not None:
            gram_train = d_transform(gram_matrix[train_index, :][:, train_index])
            gram_test = d_transform(gram_matrix[test_index, :][:, train_index])
        else:
            gram_train = gram_matrix[train_index, :][:, train_index]
            gram_test = gram_matrix[test_index, :][:, train_index]

        # Predict
        model.fit(gram_train, y[train_index])
        preds = model.predict(gram_test)
        all_preds.append(preds)
        all_true.append(y[test_index])

    return np.concatenate(all_preds), np.concatenate(all_true)

def evaluate_model(model, gram_matrix, y, gestures, nsplits=5, d_transform=None):
    nsample = len(y)
    nsample_per_group = int(nsample/nsplits)
    repeats = [nsample_per_group]*(nsplits-1)
    repeats = repeats + [nsample-np.sum(repeats)]
    groups = np.repeat(np.arange(nsplits), repeats)
    kfolder = GroupKFold(n_splits=nsplits)

    ngestures = len(gestures)
    confusion_matrix_cum = np.zeros((ngestures, ngestures))
    count_gesture_presence = np.zeros((ngestures, ngestures))
    accuracy_cum = 0
    for train_index, test_index in kfolder.split(y, groups=groups):
        if d_transform is not None:
            gram_train = d_transform(gram_matrix[train_index, :][:, train_index])
            gram_test = d_transform(gram_matrix[test_index, :][:, train_index])
        else:
            gram_train = gram_matrix[train_index, :][:, train_index]
            gram_test = gram_matrix[test_index, :][:, train_index]

        # Predict
        model.fit(gram_train, y[train_index])
        preds = model.predict(gram_test)
        accuracy_cum += np.mean(y[test_index]==preds)
        cm = confusion_matrix(y[test_index], preds, normalize='true')

        # Handle of missing gestures
        current_gesture_presence = np.ones(cm.shape)
        if cm.shape != (ngestures,ngestures):
            missing_gestures = np.setdiff1d(np.setdiff1d(gestures, preds), y[test_index])

            print(gestures, missing_gestures)
            for g in missing_gestures:
                gi = g2i[g]
                cm = np.insert(cm, gi, 0, axis=0)
                cm = np.insert(cm, gi, 0, axis=1)
                #cm[gi,gi] = 1
                print('AAA')
                print(current_gesture_presence)
                current_gesture_presence = np.insert(current_gesture_presence, gi, 0, axis=0)
                print('BBB')
                print(current_gesture_presence)
                current_gesture_presence = np.insert(current_gesture_presence, gi, 0, axis=1)
                print('CCC')
        confusion_matrix_cum += cm
        count_gesture_presence += current_gesture_presence
        print(current_gesture_presence)
    #if np.any(count_gesture_presence==0):
    #    print('In evaluate_model, zero count:')
    print('End of splits')
    print(count_gesture_presence)
    average_confusion_matrix = confusion_matrix_cum/count_gesture_presence

    accuracy = accuracy_cum / float(nsplits)
    weighted_accuracy = np.mean(np.diag(average_confusion_matrix))

    return accuracy, weighted_accuracy, average_confusion_matrix

## Define initial values and load dataset

In [15]:
import os
print(sorted(os.listdir('data/physionet'))[])

['64_channel_sharbrough-old.png', '64_channel_sharbrough.pdf', '64_channel_sharbrough.png', 'ANNOTATORS', 'RECORDS', 'S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026', 'S027', 'S028', 'S029', 'S030', 'S031', 'S032', 'S033', 'S034', 'S035', 'S036', 'S037', 'S038', 'S039', 'S040', 'S041', 'S042', 'S043', 'S044', 'S045', 'S046', 'S047', 'S048', 'S049', 'S050', 'S051', 'S052', 'S053', 'S054', 'S055', 'S056', 'S057', 'S058', 'S059', 'S060', 'S061', 'S062', 'S063', 'S064', 'S065', 'S066', 'S067', 'S068', 'S069', 'S070', 'S071', 'S072', 'S073', 'S074', 'S075', 'S076', 'S077', 'S078', 'S079', 'S080', 'S081', 'S082', 'S083', 'S084', 'S085', 'S086', 'S087', 'S088', 'S089', 'S090', 'S091', 'S092', 'S093', 'S094', 'S095', 'S096', 'S097', 'S098', 'S099', 'S100', 'S101', 'S102', 'S103', 'S104', 'S105', 'S106', 'S107', 'S108', 'S109', 'SHA256SUMS

In [2]:
ks = np.arange(1, 8)
gammas = np.logspace(-10, -5, base=2, num=10)
ncomp = 10

X,y = get_data('data/physionet/', n_classes=2)

Returning 2 Class data
Data from runs: [4, 8, 12]


OSError: data/physionet/S001/S001R04.edf: the file is not EDF(+) or BDF(+) compliant (Prefilter)

In [3]:
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
import pyedflib as edf
from save_edf import write_mne_edf

#Define the parameters 
subject = 4  # use data from subject 1
runs = [6, 10, 14]  # use only hand and feet motor imagery runs

files = [f'data/physionet/S{subject:03}/S{subject:03}R{r:02}.edf' for r in runs]
#Read raw data files where each file contains a run
raws = [read_raw_edf(f, preload=True) for f in files]
#Combine all loaded runs
raw_obj = concatenate_raws(raws)
print(type(raw_obj))

gen_file = 'data_generated/test.S4R061014.edf'
write_mne_edf(raw_obj, 'data_generated/test.S4R061014.edf', overwrite=True)

Extracting EDF parameters from /home/jules/documents/epfl/ma4/tic-detection/data/physionet/S004/S004R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/jules/documents/epfl/ma4/tic-detection/data/physionet/S004/S004R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/jules/documents/epfl/ma4/tic-detection/data/physionet/S004/S004R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
<class 'mne.io.edf.edf.RawEDF'>
saving to data_generated/test.S4R061014.edf, filetype 1


True

In [16]:
er = edf.EdfReader(gen_file)
fs = er.getSampleFrequency(0)
n_ch = er.signals_in_file
er.close()

print('Freq:', fs)
print('Number of channels:', n_ch)

Freq: 160
Number of channels: 64
